In [ ]:
import awkward as ak
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import random
import uproot

In [ ]:
f = uproot.open("dataframe_ZeroBias_20241119.root")

In [ ]:
for key in f['Events'].keys(): print(key)

In [ ]:
run = f['Events']['run'].array()
evt = f['Events']['evt'].array()
cicadaScore = f['Events']['cicadaScore'].array()
Trijet_mass = f['Events']['Trijet_mass'].array()
Trijet_eta0 = f['Events']['Trijet_eta0'].array()
Trijet_eta1 = f['Events']['Trijet_eta1'].array()
Trijet_eta2 = f['Events']['Trijet_eta2'].array()
Trijet_phi0 = f['Events']['Trijet_phi0'].array()
Trijet_phi1 = f['Events']['Trijet_phi1'].array()
Trijet_phi2 = f['Events']['Trijet_phi2'].array()
Trijet_btag0 = f['Events']['Trijet_btag0'].array()
Trijet_btag1 = f['Events']['Trijet_btag1'].array()
Trijet_btag2 = f['Events']['Trijet_btag2'].array()
muon_pt = f['Events']['muon_pt'].array()
muon_eta = f['Events']['muon_eta'].array()
muon_phi = f['Events']['muon_phi'].array()
electron_pt = f['Events']['electron_pt'].array()
electron_eta = f['Events']['electron_eta'].array()
electron_phi = f['Events']['electron_phi'].array()
jet_eta = f['Events']['jet_eta'].array()
jet_phi = f['Events']['jet_phi'].array()
jet_btagDeepCSV = f['Events']['jet_btagDeepCSV'].array()

In [ ]:
# jet colors for plotting
jetcolors = [
    'brown',
    'red',
    'orange',
    'gold',
    'forestgreen',
    'teal',
    'dodgerblue',
    'blue',
    'purple',
    'deeppink',
]
random.shuffle(jetcolors)

B_TAG_THRESHOLD = 0.5

In [ ]:
for iEvt in range(len(cicadaScore)):
    
    if ((Trijet_mass[iEvt]<165) or (Trijet_mass[iEvt]>180)): continue
    if cicadaScore[iEvt]<100: continue
    
    # define mpl figure
    fig, axs = plt.subplots(1,2,figsize=(14,6),gridspec_kw={'width_ratios': [2.3, 1]})
    
    avg_jet_eta = np.average(jet_eta[iEvt])
   
    order_elements = []
    for i in range(min(len(jet_eta[iEvt]),10)):
        # create circle centered around jet center with R=0.4
        circle = plt.Circle(
            (jet_eta[iEvt][i], jet_phi[iEvt][i]), 
            0.4, 
            facecolor=jetcolors[i], 
            edgecolor = 'k', 
            linestyle='dotted', 
            alpha=0.2
        )
        axs[0].add_patch(circle)
        
        if jet_phi[iEvt][i]>np.pi-0.4:
            circle2 = plt.Circle(
                (jet_eta[iEvt][i], jet_phi[iEvt][i]-2*np.pi), 
                0.4, 
                facecolor=jetcolors[i], 
                edgecolor = 'k', 
                linestyle='dotted', 
                alpha=0.2
            )
            axs[0].add_patch(circle2)
            
        elif jet_phi[iEvt][i]<-np.pi+0.4:
            circle2 = plt.Circle(
                (jet_eta[iEvt][i], jet_phi[iEvt][i]+2*np.pi), 
                0.4, 
                facecolor=jetcolors[i], 
                edgecolor = 'k', 
                linestyle='dotted', 
                alpha=0.2
            )
            axs[0].add_patch(circle2)
        
        
        # add border if b-tagged
        if jet_btagDeepCSV[iEvt][i]>B_TAG_THRESHOLD:
            circle = plt.Circle((jet_eta[iEvt][i], jet_phi[iEvt][i]), 0.4, facecolor=None, edgecolor = 'k', 
                                linestyle='solid', fill=False)
            if jet_phi[iEvt][i]>np.pi-0.4:
                circle2 = plt.Circle((jet_eta[iEvt][i], jet_phi[iEvt][i]-2*np.pi), 0.4, facecolor=None, edgecolor = 'k', 
                                     linestyle='solid', fill=False)
                axs[0].add_patch(circle2)
            elif jet_phi[iEvt][i]<-np.pi+0.4:
                circle2 = plt.Circle((jet_eta[iEvt][i], jet_phi[iEvt][i]+2*np.pi), 0.4, facecolor=None, edgecolor = 'k', 
                                     linestyle='solid', fill=False)
                axs[0].add_patch(circle2)
            else:
                circle2 = None
            
            axs[0].add_patch(circle)
            
        # add jet color info for legend
        order_elements.append(Patch(facecolor=jetcolors[i], edgecolor=jetcolors[i],label=''))
        
    # plot dot at center of selected jets
    axs[0].plot(
        [Trijet_eta0[iEvt], Trijet_eta1[iEvt], Trijet_eta2[iEvt]], 
        [Trijet_phi0[iEvt], Trijet_phi1[iEvt], Trijet_phi2[iEvt]],
        marker = '.',
        color = 'k',
        linestyle='none'
    )

    
    # plot muons
    current_muon_eta = np.array(muon_eta[iEvt])
    current_muon_phi = np.array(muon_phi[iEvt])
    for i in range(len(current_muon_eta)):
        if (current_muon_eta[i]>(avg_jet_eta-6)) and (current_muon_eta[i]<(avg_jet_eta+6)) and (muon_pt[iEvt][i]>5):
            axs[0].plot(current_muon_eta[i], current_muon_phi[i], 'mD')
            axs[0].text(current_muon_eta[i]+0.07, current_muon_phi[i]+0.07, "$\mu$", fontsize=10)
            
    # plot electrons
    current_electron_eta = np.array(electron_eta[iEvt])
    current_electron_phi = np.array(electron_phi[iEvt])
    for i in range(len(current_electron_eta)):
        if (current_electron_eta[i]>(avg_jet_eta-6)) and (current_electron_eta[i]<(avg_jet_eta+6)) and (electron_pt[iEvt][i]>5):
            axs[0].plot(current_electron_eta[i], current_electron_phi[i], 'mD')
            axs[0].text(current_electron_eta[i]+0.07, current_electron_phi[i]+0.07, "$\mu$", fontsize=10)
    
    
    # legend for jet order
    legend1 = plt.legend(handles=order_elements, 
                         title="Jet Order", 
                         loc='upper left')
    
    
    # legend for jets
    jet_legend_elements = [Line2D([0], [0], marker=None, color='k', label=f'b-tagged (>={B_TAG_THRESHOLD})', 
                                  linestyle='solid',alpha=1.0),
                           Line2D([0], [0], marker=None, color='k', label='not b-tagged', 
                                  linestyle='dotted',alpha=0.5),
                           Line2D([0], [0], marker='o', color='k', label='selected for top mass', linestyle='None', markersize=4)
                           ]
    
    legend3 = plt.legend(handles=jet_legend_elements, 
                         title="Jet Key", 
                         loc='lower left')
    
    # marks pi and -pi on plot
    axs[0].hlines([-np.pi, np.pi], -10, 10, color='darkgray', alpha=1.0, linewidth=1)
    
    # add legends to plot
    axs[1].add_artist(legend1)
    axs[1].add_artist(legend3)
    
    # shade areas outside phi=(-pi,pi)
    axs[0].fill_between([-10,10],np.pi,np.pi+1,color='lightgray',alpha=0.85)
    axs[0].fill_between([-10,10],-1-np.pi,-np.pi,color='lightgray',alpha=0.85)
    
    # plotting limits
    axs[0].set_xlim([avg_jet_eta-6,avg_jet_eta+6])
    axs[0].set_ylim([-3.5,3.5])
    
    # set axis labels
    axs[0].set_xlabel("$\eta$")
    axs[0].set_ylabel("$\phi$")
    
    # phi axis in units of pi
    axs[0].set_yticks([-np.pi,-3*np.pi/4,-np.pi/2,-np.pi/4,0,np.pi/4,np.pi/2,3*np.pi/4,np.pi])
    axs[0].set_yticklabels(["$-\pi$","","$-\pi/2$","","0","","$\pi/2$","","$\pi$"])
    
    axs[0].grid(alpha=0.5)
    axs[0].set_title(
        f"Event = {evt[iEvt]}, "
        f"Run = {run[iEvt]}, "
        f"CICADA Score = {np.round(cicadaScore[iEvt], decimals=1)}, "
        f"Reconstructed Top Mass = {np.round(Trijet_mass[iEvt],decimals=2)} [GeV],"
    )
    
    # make second subplot invisible (just used to get legends to show up nicely)
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['bottom'].set_visible(False)
    axs[1].spines['left'].set_visible(False)
    axs[1].get_xaxis().set_ticks([])
    axs[1].get_yaxis().set_ticks([])
    
    fig.tight_layout()
    
    plt.show()